## Attempt 1: Create Less Buckets When Preprocessing

For the first attempt, I will leave the parameters of the model the same but create far fewer buckets on the categorical variables we had, to see if less categories help.

In [28]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [29]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace = True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [30]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [31]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

I am going to alter the below code to create two types of applications: T3 and non-T3

In [32]:
# Determine which values to replace if counts are less than 500
X = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(X[X < 20000].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"NON-T3")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3        27037
NON-T3     7262
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

Similr to the last, I am going to make application type C1000 and non-C1000

In [34]:
# Determine which values to replace if counts are less than ..?
Y = application_df['CLASSIFICATION'].value_counts()
replace_class = list(Y[Y < 15000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"NON-C1000")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000        17326
NON-C1000    16973
Name: CLASSIFICATION, dtype: int64

In [35]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,NON-T3,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,NON-C1000,Preservation,Co-operative,1,1-9999,N,108590,1
2,NON-T3,CompanySponsored,NON-C1000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,NON-C1000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [36]:
# Generate our categorical variable lists
application_cat = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT',
                  'SPECIAL_CONSIDERATIONS']

In [37]:
#Create an encode and non-encode data frame
application_df_enc = application_df.copy()
application_df_enc.drop(columns=['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL'], inplace=True)

In [38]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df_enc))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\Christopher\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_NON-T3,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_NON-C1000,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [39]:
# Merge one-hot encoded features and drop the originals
application_df = pd.concat([application_df, encode_df], axis=1)

application_df.drop(columns=application_cat, inplace=True)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_NON-T3,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [40]:
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_NON-T3,APPLICATION_TYPE_T3,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [41]:
# Split our preprocessed data into our features and target arrays
X = application_df.copy()
X.drop(columns = ['IS_SUCCESSFUL'], inplace=True)

y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=573, stratify=y)

In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2640      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
_________________________________________________________________


In [44]:
#Create a callback that saves the model's weights every 5 epochs

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [45]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [46]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    periods=5)

In [47]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=30,callbacks=[cp_callback])

Epoch 1/30
793/804 [============================>.] - ETA: 0s - loss: 46148.3516 - accuracy: 0.4931
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 47326.7617 - accuracy: 0.4931
Epoch 2/30
785/804 [============================>.] - ETA: 0s - loss: 20571.0195 - accuracy: 0.4945
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 20096.0918 - accuracy: 0.4949
Epoch 3/30
793/804 [============================>.] - ETA: 0s - loss: 19313.2109 - accuracy: 0.4994
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 19260.1738 - accuracy: 0.4990
Epoch 4/30
800/804 [============================>.] - ETA: 0s - loss: 17551.2754 - accuracy: 0.4920
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 17780.5039 - accuracy: 0.4919
Epoch 5/30
787/8

In [49]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.9109 - accuracy: 0.4515 - 387ms/epoch - 1ms/step
Loss: 0.9109473824501038, Accuracy: 0.45154517889022827


## Attempt 2: Modify Model Parameters

Reducing complexity of the data caused accuracy to go down, so I will go back to the prior processing method and instead look at model attributes to increase accuracy.  In this case I will try to increase the number of nodes on the two hidden layers.

In [50]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")

In [51]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace = True)

In [52]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [53]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [54]:
# Determine which values to replace if counts are less than 500
X = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(X[X < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"NON-T3")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3        27037
T4         1542
T6         1216
T5         1173
T19        1065
T8          737
T7          725
T10         528
NON-T3      276
Name: APPLICATION_TYPE, dtype: int64

In [55]:
# Determine which values to replace if counts are less than ..?
Y = application_df['CLASSIFICATION'].value_counts()
replace_class = list(Y[Y < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"NON-C1000")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000        17326
C2000         6074
C1200         4837
NON-C1000     2261
C3000         1918
C2100         1883
Name: CLASSIFICATION, dtype: int64

In [56]:
# Generate our categorical variable lists
application_cat = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT',
                  'SPECIAL_CONSIDERATIONS']

In [57]:
#Create an encode and non-encode data frame
application_df_enc = application_df.copy()
application_df_enc.drop(columns=['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL'], inplace=True)

In [58]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df_enc))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

C:\Users\Christopher\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [59]:
# Merge one-hot encoded features and drop the originals
application_df = pd.concat([application_df, encode_df], axis=1)

application_df.drop(columns=application_cat, inplace=True)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_NON-T3,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [60]:
# Split our preprocessed data into our features and target arrays
X = application_df.copy()
X.drop(columns = ['IS_SUCCESSFUL'], inplace=True)

y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=573, stratify=y)

In [61]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [62]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               4400      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 9,501
Trainable params: 9,501
Non-trainable params: 0
_________________________________________________________________


In [63]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [64]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [65]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    periods=5)

In [66]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=30,callbacks=[cp_callback])

Epoch 1/30
791/804 [============================>.] - ETA: 0s - loss: 54121.5000 - accuracy: 0.4839
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 3s 2ms/step - loss: 53263.5742 - accuracy: 0.4836
Epoch 2/30
786/804 [============================>.] - ETA: 0s - loss: 74818.2891 - accuracy: 0.4865
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 73754.3359 - accuracy: 0.4862
Epoch 3/30
782/804 [============================>.] - ETA: 0s - loss: 19375.1367 - accuracy: 0.5078
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 19162.5918 - accuracy: 0.5073
Epoch 4/30
801/804 [============================>.] - ETA: 0s - loss: 28370.4395 - accuracy: 0.4956
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 28274.4473 - accuracy: 0.4954
Epoch 5/30
787/8

In [67]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.9306 - accuracy: 0.6928 - 491ms/epoch - 2ms/step
Loss: 0.9305688142776489, Accuracy: 0.6928279995918274


## Attempt 3: Increase Number of Nodes More

By increasing the number of nodes, accuracy went from 56% (baseline from Deliverable 2) to 69%.  I am going to further increase the number of nodes and see if I continue to see an increase.

In [68]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")

In [69]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace = True)

In [70]:
# Determine which values to replace if counts are less than 500
X = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(X[X < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [71]:
# Determine which values to replace if counts are less than ..?
Y = application_df['CLASSIFICATION'].value_counts()
replace_class = list(Y[Y < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [72]:
# Generate our categorical variable lists
application_cat = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT',
                  'SPECIAL_CONSIDERATIONS']

In [73]:
#Create an encode and non-encode data frame
application_df_enc = application_df.copy()
application_df_enc.drop(columns=['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL'], inplace=True)

In [74]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df_enc))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

C:\Users\Christopher\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [75]:
# Merge one-hot encoded features and drop the originals
application_df = pd.concat([application_df, encode_df], axis=1)

application_df.drop(columns=application_cat, inplace=True)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [76]:
# Split our preprocessed data into our features and target arrays
X = application_df.copy()
X.drop(columns = ['IS_SUCCESSFUL'], inplace=True)

y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=573, stratify=y)

In [77]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [78]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 1000
hidden_nodes_layer2 = 150

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1000)              44000     
                                                                 
 dense_7 (Dense)             (None, 150)               150150    
                                                                 
 dense_8 (Dense)             (None, 1)                 151       
                                                                 
Total params: 194,301
Trainable params: 194,301
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [80]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [81]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    periods=5)

In [82]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=30,callbacks=[cp_callback])

Epoch 1/30
800/804 [============================>.] - ETA: 0s - loss: 5751.8652 - accuracy: 0.5182
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 5s 4ms/step - loss: 5724.1411 - accuracy: 0.5178
Epoch 2/30
794/804 [============================>.] - ETA: 0s - loss: 0.6915 - accuracy: 0.5318
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.6915 - accuracy: 0.5324
Epoch 3/30
795/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5329
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 4/30
797/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5324
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 5/30
795/804 [======================

In [83]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6864 - accuracy: 0.5619 - 545ms/epoch - 2ms/step
Loss: 0.6863518357276917, Accuracy: 0.5618658661842346


## Attempt 4: Add a Third Layer

Further increasing the nodes on the two layers beyond attempt 2 caused accuracy to go down.  Now I will attempt to add a third layer and see if that helps.

In [84]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")

In [85]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace = True)

In [86]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [87]:
# Determine which values to replace if counts are less than 500
X = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(X[X < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"OTHER")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
OTHER      276
Name: APPLICATION_TYPE, dtype: int64

In [88]:
# Determine which values to replace if counts are less than ..?
Y = application_df['CLASSIFICATION'].value_counts()
replace_class = list(Y[Y < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"OTHER")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
OTHER     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [89]:
# Generate our categorical variable lists
application_cat = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT',
                  'SPECIAL_CONSIDERATIONS']

In [90]:
#Create an encode and non-encode data frame
application_df_enc = application_df.copy()
application_df_enc.drop(columns=['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL'], inplace=True)

In [91]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df_enc))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

C:\Users\Christopher\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [92]:
# Merge one-hot encoded features and drop the originals
application_df = pd.concat([application_df, encode_df], axis=1)

application_df.drop(columns=application_cat, inplace=True)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_OTHER,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [93]:
# Split our preprocessed data into our features and target arrays
X = application_df.copy()
X.drop(columns = ['IS_SUCCESSFUL'], inplace=True)

y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=573, stratify=y)

In [94]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [95]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 100)               4400      
                                                                 
 dense_10 (Dense)            (None, 50)                5050      
                                                                 
 dense_11 (Dense)            (None, 20)                1020      
                                                                 
 dense_12 (Dense)            (None, 1)                 21        
                                                                 
Total params: 10,491
Trainable params: 10,491
Non-trainable params: 0
_________________________________________________________________


In [96]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [97]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [98]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    periods=5)

In [99]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=30,callbacks=[cp_callback])

Epoch 1/30
779/804 [============================>.] - ETA: 0s - loss: 36739.6914 - accuracy: 0.4955
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 3s 2ms/step - loss: 44892.0508 - accuracy: 0.4948
Epoch 2/30
783/804 [============================>.] - ETA: 0s - loss: 13579.9082 - accuracy: 0.4943
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 13322.2344 - accuracy: 0.4935
Epoch 3/30
796/804 [============================>.] - ETA: 0s - loss: 8093.9160 - accuracy: 0.4912
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 8077.4585 - accuracy: 0.4919
Epoch 4/30
789/804 [============================>.] - ETA: 0s - loss: 12484.2383 - accuracy: 0.5093
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 12265.4189 - accuracy: 0.5091
Epoch 5/30
787/804

In [100]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6740 - accuracy: 0.5978 - 479ms/epoch - 2ms/step
Loss: 0.674040675163269, Accuracy: 0.5977842807769775


In [94]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Attempt 5: Modifications of Attempt 2
Trying different activation functions.

In [137]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="leaky_relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="leaky_relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 100)               4400      
                                                                 
 dense_33 (Dense)            (None, 50)                5050      
                                                                 
 dense_34 (Dense)            (None, 1)                 51        
                                                                 
Total params: 9,501
Trainable params: 9,501
Non-trainable params: 0
_________________________________________________________________


In [138]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [139]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [140]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    periods=5)

In [141]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
803/804 [============================>.] - ETA: 0s - loss: 39829.4492 - accuracy: 0.4966
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 3s 2ms/step - loss: 39786.0977 - accuracy: 0.4967
Epoch 2/100
795/804 [============================>.] - ETA: 0s - loss: 27920.5566 - accuracy: 0.5056
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 28435.5352 - accuracy: 0.5051
Epoch 3/100
779/804 [============================>.] - ETA: 0s - loss: 41712.3789 - accuracy: 0.4932
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 40437.0938 - accuracy: 0.4950
Epoch 4/100
786/804 [============================>.] - ETA: 0s - loss: 16406.8262 - accuracy: 0.4937
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 16043.8145 - accuracy: 0.4950
Epoch 5/100


780/804 [============================>.] - ETA: 0s - loss: 158.1458 - accuracy: 0.5151
Epoch 34: saving model to checkpoints\weights.34.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 153.9780 - accuracy: 0.5152
Epoch 35/100
783/804 [============================>.] - ETA: 0s - loss: 588.4104 - accuracy: 0.5126
Epoch 35: saving model to checkpoints\weights.35.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 575.0231 - accuracy: 0.5138
Epoch 36/100
790/804 [============================>.] - ETA: 0s - loss: 66.9937 - accuracy: 0.5211
Epoch 36: saving model to checkpoints\weights.36.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 69.6416 - accuracy: 0.5219
Epoch 37/100
790/804 [============================>.] - ETA: 0s - loss: 390.3975 - accuracy: 0.5117
Epoch 37: saving model to checkpoints\weights.37.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 396.4341 - accuracy: 0.5110
Epoch 38/100
803/804 [=============

804/804 [==============================] - 2s 2ms/step - loss: 496.8389 - accuracy: 0.5001
Epoch 68/100
803/804 [============================>.] - ETA: 0s - loss: 96.7928 - accuracy: 0.5183
Epoch 68: saving model to checkpoints\weights.68.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 105.3569 - accuracy: 0.5184
Epoch 69/100
803/804 [============================>.] - ETA: 0s - loss: 59.0329 - accuracy: 0.5238
Epoch 69: saving model to checkpoints\weights.69.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 58.9694 - accuracy: 0.5239
Epoch 70/100
784/804 [============================>.] - ETA: 0s - loss: 182.8349 - accuracy: 0.5216
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 179.9097 - accuracy: 0.5215
Epoch 71/100
785/804 [============================>.] - ETA: 0s - loss: 197.6322 - accuracy: 0.5100
Epoch 71: saving model to checkpoints\weights.71.hdf5
804/804 [==============

In [142]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8838 - accuracy: 0.5324 - 482ms/epoch - 2ms/step
Loss: 0.8838361501693726, Accuracy: 0.5323615074157715


## Attempt 6: Adding many more layers
It looks like relu had better results than leaky relu, going back to try more layers and nodes with relu.

In [143]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50
hidden_nodes_layer4 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 100)               4400      
                                                                 
 dense_36 (Dense)            (None, 100)               10100     
                                                                 
 dense_37 (Dense)            (None, 50)                5050      
                                                                 
 dense_38 (Dense)            (None, 50)                2550      
                                                                 
 dense_39 (Dense)            (None, 1)                 51        
                                                                 
Total params: 22,151
Trainable params: 22,151
Non-trainable params: 0
_________________________________________________________________


In [144]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [145]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [146]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    periods=5)

In [147]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
790/804 [============================>.] - ETA: 0s - loss: 15048.6670 - accuracy: 0.4856
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 3s 2ms/step - loss: 14952.1348 - accuracy: 0.4860
Epoch 2/100
797/804 [============================>.] - ETA: 0s - loss: 6554.3125 - accuracy: 0.4884
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 6499.1025 - accuracy: 0.4881
Epoch 3/100
790/804 [============================>.] - ETA: 0s - loss: 6647.1616 - accuracy: 0.4960
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 6538.8618 - accuracy: 0.4960
Epoch 4/100
781/804 [============================>.] - ETA: 0s - loss: 1837.0999 - accuracy: 0.4921
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 1794.7748 - accuracy: 0.4920
Epoch 5/100
787/80

Epoch 35/100
801/804 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5325
Epoch 35: saving model to checkpoints\weights.35.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6914 - accuracy: 0.5324
Epoch 36/100
799/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5321
Epoch 36: saving model to checkpoints\weights.36.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 37/100
786/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5321
Epoch 37: saving model to checkpoints\weights.37.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 38/100
783/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5312
Epoch 38: saving model to checkpoints\weights.38.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5313
Epoch 39/100
791/804 [==============

Epoch 69/100
790/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5318
Epoch 69: saving model to checkpoints\weights.69.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6910 - accuracy: 0.5324
Epoch 70/100
802/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5323
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 71/100
800/804 [============================>.] - ETA: 0s - loss: 0.6909 - accuracy: 0.5328
Epoch 71: saving model to checkpoints\weights.71.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6910 - accuracy: 0.5324
Epoch 72/100
799/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5325
Epoch 72: saving model to checkpoints\weights.72.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 73/100
791/804 [==============

In [148]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6935 - accuracy: 0.5315 - 504ms/epoch - 2ms/step
Loss: 0.6934595704078674, Accuracy: 0.5315451622009277


## Attempt 7: Three Layers, Less Nodes

In [260]:
number_input_features

43

In [261]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 43
hidden_nodes_layer2 = 21
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_145 (Dense)           (None, 43)                1892      
                                                                 
 dense_146 (Dense)           (None, 21)                924       
                                                                 
 dense_147 (Dense)           (None, 10)                220       
                                                                 
 dense_148 (Dense)           (None, 1)                 11        
                                                                 
Total params: 3,047
Trainable params: 3,047
Non-trainable params: 0
_________________________________________________________________


In [262]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [263]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [264]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    periods=5)

In [265]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=20,callbacks=[cp_callback])

Epoch 1/20
780/804 [============================>.] - ETA: 0s - loss: 30825.2148 - accuracy: 0.4901
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 3s 2ms/step - loss: 29943.1973 - accuracy: 0.4900
Epoch 2/20
784/804 [============================>.] - ETA: 0s - loss: 11844.3848 - accuracy: 0.4990
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 11565.5439 - accuracy: 0.4985
Epoch 3/20
781/804 [============================>.] - ETA: 0s - loss: 10611.5029 - accuracy: 0.5074
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 10654.2500 - accuracy: 0.5064
Epoch 4/20
793/804 [============================>.] - ETA: 0s - loss: 16301.9521 - accuracy: 0.4931
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 16602.7520 - accuracy: 0.4934
Epoch 5/20
797/8

In [266]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.8039 - accuracy: 0.5313 - 525ms/epoch - 2ms/step
Loss: 0.8038981556892395, Accuracy: 0.5313119292259216


## Attempt 8: More Data Cleanup

In [430]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")

In [431]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace = True)

In [432]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [433]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [434]:
# Determine which values to replace if counts are less than 500
X = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(X[X < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"OTHER")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
OTHER      276
Name: APPLICATION_TYPE, dtype: int64

In [435]:
# Determine which values to replace if counts are less than ..?
Y = application_df['CLASSIFICATION'].value_counts()
replace_class = list(Y[Y < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"OTHER")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
OTHER     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [436]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [437]:
application_df.ORGANIZATION.value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [438]:
# Determine which values to replace if counts are less than ..?
Y = application_df['ORGANIZATION'].value_counts()
replace_class = list(Y[Y < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.ORGANIZATION = application_df.ORGANIZATION.replace(cls,"OTHER")
    
# Check to make sure binning was successful
application_df.ORGANIZATION.value_counts()

Trust          23515
Association    10255
OTHER            529
Name: ORGANIZATION, dtype: int64

In [439]:
application_df.USE_CASE.value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [440]:
# Determine which values to replace if counts are less than ..?
Y = application_df['USE_CASE'].value_counts()
replace_class = list(Y[Y < 5000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.USE_CASE = application_df.USE_CASE.replace(cls,"OTHER")
    
# Check to make sure binning was successful
application_df.ORGANIZATION.value_counts()

Trust          23515
Association    10255
OTHER            529
Name: ORGANIZATION, dtype: int64

In [441]:
Z = application_df.INCOME_AMT.value_counts()

In [442]:
# Determine which values to replace if counts are less than ..?
replace_class = list(Z[Z < 900].index)

# Replace in dataframe
for cls in replace_class:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"OTHER")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
OTHER             1835
1M-5M              955
Name: INCOME_AMT, dtype: int64

In [443]:
len(application_df.index)

34299

In [444]:
application_df.ASK_AMT.value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [445]:
application_df.STATUS.value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [446]:
application_df.drop(columns=['ASK_AMT', 'STATUS'], inplace=True)

In [447]:
# Generate our categorical variable lists
application_cat = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT',
                  'SPECIAL_CONSIDERATIONS']

In [448]:
#Create an encode and non-encode data frame
application_df_enc = application_df.copy()
application_df_enc.drop(columns=['IS_SUCCESSFUL'], inplace=True)

In [449]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df_enc))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

C:\Users\Christopher\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [450]:
# Merge one-hot encoded features and drop the originals
application_df = pd.concat([application_df, encode_df], axis=1)

application_df.drop(columns=application_cat, inplace=True)
application_df.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_OTHER,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Association,ORGANIZATION_OTHER,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_OTHER,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [451]:
# Split our preprocessed data into our features and target arrays
X = application_df.copy()
X.drop(columns = ['IS_SUCCESSFUL'], inplace=True)

y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=573, stratify=y)

In [452]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [453]:
len(X.columns)

34

In [454]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_172 (Dense)           (None, 80)                2800      
                                                                 
 dense_173 (Dense)           (None, 30)                2430      
                                                                 
 dense_174 (Dense)           (None, 1)                 31        
                                                                 
Total params: 5,261
Trainable params: 5,261
Non-trainable params: 0
_________________________________________________________________


In [455]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [456]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [457]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    periods=5)

In [458]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=10,callbacks=[cp_callback])

Epoch 1/10
787/804 [============================>.] - ETA: 0s - loss: 0.5755 - accuracy: 0.7173
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5750 - accuracy: 0.7176
Epoch 2/10
801/804 [============================>.] - ETA: 0s - loss: 0.5596 - accuracy: 0.7260
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5599 - accuracy: 0.7258
Epoch 3/10
778/804 [============================>.] - ETA: 0s - loss: 0.5570 - accuracy: 0.7258
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5572 - accuracy: 0.7253
Epoch 4/10
782/804 [============================>.] - ETA: 0s - loss: 0.5555 - accuracy: 0.7270
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5555 - accuracy: 0.7271
Epoch 5/10
799/804 [============================

In [459]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.9222 - accuracy: 0.7111 - 511ms/epoch - 2ms/step
Loss: 0.9222463369369507, Accuracy: 0.7111369967460632


## Attempt 9: Trials with Cleaner Data

In [460]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_175 (Dense)           (None, 50)                1750      
                                                                 
 dense_176 (Dense)           (None, 25)                1275      
                                                                 
 dense_177 (Dense)           (None, 1)                 26        
                                                                 
Total params: 3,051
Trainable params: 3,051
Non-trainable params: 0
_________________________________________________________________


In [461]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [462]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [463]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    periods=5)

In [464]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=20,callbacks=[cp_callback])

Epoch 1/20
788/804 [============================>.] - ETA: 0s - loss: 0.5813 - accuracy: 0.7139
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 3s 2ms/step - loss: 0.5809 - accuracy: 0.7142
Epoch 2/20
786/804 [============================>.] - ETA: 0s - loss: 0.5597 - accuracy: 0.7254
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5602 - accuracy: 0.7248
Epoch 3/20
799/804 [============================>.] - ETA: 0s - loss: 0.5579 - accuracy: 0.7259
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5576 - accuracy: 0.7260
Epoch 4/20
780/804 [============================>.] - ETA: 0s - loss: 0.5550 - accuracy: 0.7267
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5554 - accuracy: 0.7266
Epoch 5/20
798/804 [============================

In [465]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.1053 - accuracy: 0.6119 - 487ms/epoch - 2ms/step
Loss: 1.105265498161316, Accuracy: 0.6118950247764587


In [466]:
#Export to H5 file
nn.save("AlphabetSoupCharity_Optimization.h5")